In [1]:
from huggingface_hub import login
import os
from google.colab import userdata

hf_token = userdata.get('HUGGINGFACEHUB_API_TOKEN')

# Ensure the token is set as an environment variable
if hf_token:
  os.environ['HUGGINGFACEHUB_API_TOKEN'] = hf_token
  # login(token = hf_token)
def check_env():
  required = "HUGGINGFACEHUB_API_TOKEN"
  if required not in os.environ:
    raise EnvironmentError(
        f"{required} not found. Please add it via colab Secrets."
    )
  else:
    print("Token present for HUGGINGFACE")

check_env()

Token present for HUGGINGFACE


In [2]:
! pip install langchain-huggingface

## Basic model Invoke and test

In [3]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

model = HuggingFaceEndpoint(
    repo_id = "meta-llama/Llama-3.3-70B-Instruct",
    task = "text-generation",
    max_new_tokens = 512,
    do_sample = False,
    repetition_penalty = 1.03,
)

chat = ChatHuggingFace(llm = model)

In [4]:
message = [{
    "role": "system",
    "content": " you are a helpful AI Assistant"
}]

message = "what is the largest island on this world? "
response = chat.invoke(message)
print(response.content)

HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/groq/openai/v1/chat/completions (Request ID: Root=1-6968ce7c-705a2d5b3f2b22ca200034be;1946c697-37ea-4f35-b7d8-a3a5c6427bce)

You have reached the free monthly usage limit for groq. Subscribe to PRO to get 20x more included usage, or add pre-paid credits to your account.

## Using Different Message styles

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage,AIMessage

messages = [
    SystemMessage(content = "you are a chef with sense of humor"),
    HumanMessage(content = "help me cook tea Indian style in Hindi!")
]

response = chat.invoke(messages)
print(response.content)

## Email prompt Templates

In [ ]:
from langchain_core.prompts import PromptTemplate

email_template = PromptTemplate.from_template(
  "Create an invitation email to the recipient that is {recipient_name}\
  for an event that is {event_type}\
  in a language that is {language}\
  Mention the event location that is {event_location}\
  and event date that is {event_date}.\
  Also write few sentences about the event description that is {event_description}\
  in style that is {style}.\
  Sender is {sender}"
)

details = {
  "recipient_name":"Meetu",
  "event_type":"Health Check",
  "language": "Indian Hindi",
  "event_location":"sarkari hospital, ghantaghar",
  "event_date":"11 AM, January 15, 2026",
  "event_description":"full body check up camp for free",
  "style":"enthusiastic tone",
  "sender": "CMO"
}


prompt = email_template.invoke(details)
print(prompt)

In [ ]:
response = chat.invoke(prompt)
print(response.content)

## Parsing of response

#### Date time Parser with Pydantic

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser



class parsedOutput(BaseModel):
  datetime: str = Field(description = "ISO 8601 datetime string")


parser_date_time = PydanticOutputParser(pydantic_object =parsedOutput )
# model_with_structure = model.with_structured_output(parsedOutput)
prompt_datetime = PromptTemplate.from_template(
    template = ("Answer the question.\n"
              "{format_instructions}\n"
              "{question}"
              ),
    # input_variables = ["question"] ,
    partial_variables = {"format_instructions" : parser_date_time.get_format_instructions()}

)


prompt_value = prompt_datetime.invoke({"question": "When was langchain released?"})

response = chat.invoke(prompt_value)

print(response.content)
returned_object = parser_date_time.parse(response.content)
print(type(returned_object))

### CSV Parser

In [ ]:
from typing import List


class CSVParser(BaseModel):
  csvoutput: List[str] = Field(description = "Comma Separated values")

parser_csv = PydanticOutputParser(pydantic_object = CSVParser)

prompt_csv = PromptTemplate.from_template(
    template = ("Answer the question.\n"
              "{format_instructions}\n"
              "{question}"
              ),
    # input_variables = ["question"] ,
    partial_variables = {"format_instructions" : parser_csv.get_format_instructions()}

)
prompt_value = prompt_csv.invoke({"question": "What are the main important frameworks for building RAG system ?"})

response = chat.invoke(prompt_value)

print(response.content)
returned_object = parser_csv.parse(response.content)
print(type(returned_object))



### Parsing thorugh .with_structured_output()

In [ ]:
class Author(BaseModel):
    name: str = Field(description="The name of the author as a string.")
    number: int = Field(description="The number of books as an INTEGER, not a string.")
    books: list[str] = Field(description="A JSON ARRAY of book titles (strings). Do NOT return a stringified array.")



# structured_llm = chat.with_structured_output(Author)

# returned_object = structured_llm.invoke(
#     "Generate the books written by Dan Brown. "
#     "Return 'number' as an integer (not a string) and 'books' as a JSON array of strings (not a quoted string)."
# )

# print(f"{returned_object.name} wrote {returned_object.number} books.")
# print(returned_object.books)


# #Throws error because Huggingface doesn't support function calling.
# #It can be done through openAI, Azure

### Chaining

In [ ]:
parser_author = PydanticOutputParser(pydantic_object=Author)
prompt_template = PromptTemplate.from_template(
    template = ("Answer the question.\n"
              "{format_instructions}\n"
              "{question}"
              ),
    partial_variables = {"format_instructions" : parser_author.get_format_instructions()}
)

chain = prompt_template | chat | parser_author

returned_object = chain.invoke({"question": "Generate the books written by Munshi Premchand in Hindi!"})

print(f"{returned_object.name} wrote {returned_object.number} books.")
print(returned_object.books)


### Runnable Lambda for chaining

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda


parse_template = PromptTemplate(
    input_variables = ["raw_feedback"],
    template = "Parse and clean the following customer feedback for key information: \n\n {raw_feedback}"
)

summary_template = PromptTemplate(
    input_variables = ["clean_feedback"],
    template = "Summarize the following customer feedback in one sentence: \n\n {clean_feedback}"
)


sentiment_template = PromptTemplate(
    input_variables = ["feedback"],
    template = "What is the sentiment of the following customer feedback? \n\n {feedback}"
)

format_cleaned_output = RunnableLambda(lambda output: {"clean_feedback": output})

format_summary_output = RunnableLambda(lambda output:{"feedback": output})

user_feedback = "The course was long, and the language was difficult to comprehend. However, the practical coding helped it to understand quickly."

chain = parse_template | chat | format_cleaned_output | summary_template | chat | format_summary_output | sentiment_template | chat | StrOutputParser()

feedback_sentiment = chain.invoke({"raw_feedback": user_feedback})

print(feedback_sentiment)

returned_object = chain.invoke(user_feedback)

print(returned_object)



### RunnableLambda Custome functions and Conditional Routing

In [6]:
!pip install langchain_groq

In [4]:
from langchain_groq import ChatGroq
import os
from google.colab import userdata
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate



os.environ["GROQ_API_KEY"] = userdata.get("groq_api_key")

chat = ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct")

parse_template = PromptTemplate(
    input_variables = ["raw_feedback"],
    template = "Parse and clean the following customer feedback for key information: \n\n {raw_feedback}"
)

summary_template = PromptTemplate(
    input_variables = ["clean_feedback"],
    template = "Summarize the following customer feedback in one sentence: \n\n {clean_feedback}"
)


sentiment_template = PromptTemplate(
    input_variables = ["feedback"],
    template = "What is the sentiment of the following customer feedback? \n\n {feedback}"
)


thankyou_template = PromptTemplate(
    input_variables=["feedback"],
    template="Given the feedback, draft a thank you message for the user and request them to leave a positive rating on our webpage:\n\n{feedback}"
)

details_template = PromptTemplate(
    input_variables=["feedback"],
    template="Given the feedback, draft a message for the user and request them provide more details about their concern:\n\n{feedback}"
)

apology_template = PromptTemplate(
    input_variables=["feedback"],
    template="Given the feedback, draft an apology message for the user and mention that their concern has been forwarded to the relevant department:\n\n{feedback}"
)
user_feedback = "The course was long, and the language was difficult to comprehend. However, the practical coding helped it to understand quickly."


thank_you_chain = thankyou_template | chat | StrOutputParser()
details_chain = details_template | chat | StrOutputParser()
apology_chain = apology_template | chat | StrOutputParser()



def route_chain(info):
  if "positive" in info["sentiment"]:
    return thankyou_template
  elif "negative" in info["sentiment"]:
    return apology_template
  else:
    return details_template



format_parsed_output = RunnableLambda(lambda output: {"clean_feedback": output})

summary_chain = parse_template | chat | format_parsed_output | summary_template | chat | StrOutputParser()

sentiment_chain = sentiment_template | chat | StrOutputParser()

summary = summary_chain.invoke({"raw_feedback": user_feedback})
sentiment = sentiment_chain.invoke({"feedback": summary})

print(f"{summary} is the summary of the feedback.\n\n {sentiment} is the sentiment of the feedback.")

full_chain = {"feedback": lambda x:x["feedback"],
              "sentiment": lambda x: x["sentiment"]
              } | RunnableLambda(route_chain)

print(full_chain.invoke({"feedback": summary, "sentiment": sentiment}))




The customer found the course to be too long with difficult language, but appreciated the practical coding exercises, suggesting that revisions should focus on conciseness, simplified language, and continued inclusion of hands-on coding activities. is the summary of the feedback.

 The sentiment of the customer feedback is **Mixed** or **Constructive Criticism**.

The customer expresses both positive and negative sentiments:

* Negative: The course is too long and uses difficult language, which are criticisms.
* Positive: The customer appreciates the practical coding exercises, which suggests that some aspects of the course are valuable.

The overall tone is constructive, as the customer provides specific suggestions for improvement, indicating a desire to help the course provider refine their product rather than simply expressing frustration or disappointment. is the sentiment of the feedback.
text='Given the feedback, draft a thank you message for the user and request them to leave a